### Import Modules

In [4]:
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.features import ColorLine
import rasterstats
import json

# Initialize map

In [237]:
# Determine Map center
route_45_center = route_45.mean()
route_45_center_coord = [route_45_center.values[0], route_45_center.values[1]]

#Create the map
m = folium.Map(location = route_45_center_coord, zoom_start = 12)
m

# Read shp file 

In [68]:
fp = '../data/Transit_Routes_for_King_County_Metro__transitroute_line.shp'
data = gpd.read_file(fp)
dat = data[data['IN_SERVICE']=='Y']
da = dat[data['ROUTE_NUM'] == 45]


,OBJECTID,CHANGE_NUM,MINOR_CHAN,CURRENT_NE,IN_SERVICE,ROUTE_ID,LOCAL_EXPR,ROUTE_NUM,SHAPE_Leng,geometry
0,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,(LINESTRING (-122.3075605352953 47.65053260749...


# break down multi linestring to single linestring

In [82]:
import fiona
from shapely.geometry import  MultiLineString, mapping, shape
with fiona.open('da.shp') as source:
    # create the new shapefile
    with fiona.open('lines.shp','w', driver='ESRI Shapefile',crs=source.crs,schema=source.schema) as ouput:
        # iterate the features of the original shapefile
        for elem in source:
               # iterate the list of geometries in one element (split the MultiLineString)
            for line in shape(elem['geometry']): 
                # write the line to the new shapefile
                ouput.write({'geometry':mapping(line),'properties':elem['properties']})

In [229]:
r = gpd.read_file('lines.shp')
R = r.to_json()
r45 = folium.GeoJson(R)
m.add_child(r45)
m

# Try to get elevation

In [232]:
sh= rasterstats.zonal_stats('lines.shp', '../data/kc_66_proj3.tif')
SH = pd.DataFrame(sh)
r['max'] = SH['max']
r

,OBJECTID,CHANGE_NUM,MINOR_CHAN,CURRENT_NE,IN_SERVICE,ROUTE_ID,LOCAL_EXPR,ROUTE_NUM,SHAPE_Leng,geometry,max
0,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3075605352953 47.650532607492...,99.969002
1,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3075605352953 47.650532607492...,119.307007
2,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3107275034277 47.652094806596...,102.046722
3,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3115149015316 47.652484919657...,100.254448
4,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3115149015316 47.652484919657...,100.451881
5,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3124628890345 47.652984815362...,89.578796
6,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3132232503292 47.653393638881...,89.578796
7,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3132232503292 47.653393638881...,140.569778
8,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3132461109814 47.656834269145...,184.838211
9,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3132070900588 47.658314904223...,184.838211


In [233]:
R = r.to_json()
r45 = folium.GeoJson(R,style_function=style_function)
m.add_child(r45)
m

# below are pointless for now

In [262]:
import branca

colorscale = branca.colormap.linear.YlOrRd_09.scale(SH['max'].min(),SH['max'].max())


def style_function(feature):
    return {
        'fillOpacity': 0.5,
        'weight': 3,
        'fillColor': '#black' if SH['max'] is None else colorscale(SH['max'])
    }

In [267]:
R = r.to_json()
r45 = folium.GeoJson(R,style_function=style_function).add_to(m)
m

TypeError: '<=' not supported between instances of 'NoneType' and 'float'

# Try convert single line segment to points

with fiona.open('lines.shp') as source:
    # create the new shapefile
    with fiona.open('points.shp','w', driver='ESRI Shapefile',crs=source.crs,schema=source.schema) as ouput:
        # iterate the features of the original shapefile
        for elem in source:
               # iterate the list of geometries in one element (split the MultiLineString)
            for line in shape(elem['geometry']): 
                # write the line to the new shapefile
                ouput.write({'geometry':mapping(line),'properties':elem['properties']})
gpd.read_file('points.shp')


In [99]:
r45 = folium.GeoJson(json.dumps(sh))
route_45_map.add_child(r45)
route_45_map

AttributeError: 'list' object has no attribute 'keys'

# Geojson Map

In [123]:
import branca.colormap as cm
import math

f2 = folium.Figure(height = 400)

max_mph = math.ceil(gdf_line['mph'].max())
linear = cm.linear.YlOrRd.scale(0,max_mph)

NameError: name 'gdf_line' is not defined

# Color function

In [119]:
def alt_color(route_45_z):
    colors = ['#0000FF', '#0040FF', '#0080FF', '#00FFB0', '#00E000',
              '#80FF00', '#FFFF00', '#FFC000', '#FF0000']
    route_alt_range = np.linspace(min(route_45_z), max(route_45_z), len(colors))
    color_alt = []
    for idx in range(len(route_45_z) - 1):
        alt_1 = route_45_z[idx]
        index_1 = np.where(route_alt_range <= alt_1)[0][-1]
        color_alt.append(colors[index_1])

    return color_alt

In [4]:
colors_alt = alt_color(route_45_z)
route_alt = ColorLine(route_45_xy, colors=list(range(0, len(route_45_z))),  # only to use color line instead of polyline
              colormap=colors_alt, weight=3).add_to(route_45_map)

route_45_map.fit_bounds(route_45_map.get_bounds())
route_45_map.crs

NameError: name 'route_45_z' is not defined